In [1]:
# Dependencies
import requests
import json
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
home_value_index_pd = pd.read_csv(
    "../Madison/Resources/Zillow Home Value Index (ZHVI).csv",
    dtype = "object",
    encoding = "utf-8"
)

# Display sample data
home_value_index_pd.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,9,0,California,state,NaN,189260.5804905432,189896.74179229472,190755.71532745418,192620.29075163053,194787.5788005574,...,736657.0613053188,729610.0999740328,722365.2612371506,717035.9787022044,716448.9319981743,719438.9021737709,724674.9614647272,731733.2131000826,739719.9182151351,747352.0519881115
1,54,1,Texas,state,NaN,110479.87192982272,110539.90954602743,110569.02873045664,110713.54982495937,110807.14395215838,...,302314.834633247,300624.99337619205,299018.04456711956,297945.50207625027,297575.3883943631,297895.17521786573,298724.90307306795,299472.14871574374,300052.0200432296,300089.9595431615
2,14,2,Florida,state,NaN,107306.76072497867,107539.19715756495,107820.95644586982,108393.70060905674,109011.34346932755,...,388123.3368840588,386773.08043296676,385629.718442275,385164.6835781837,385610.8495285095,386666.53037803964,388345.4437035078,390052.85717714886,391744.88612785895,392921.5945984132
3,43,3,New York,state,NaN,153890.06182166748,154438.06160954604,154965.59977878895,156100.19097092343,157274.08727433413,...,441114.3983949437,439438.4457567081,438046.0689251003,438141.3533454628,439650.7306595852,442121.0596610177,445024.4569522553,448103.1301720137,451232.0242803551,453887.25695120654
4,47,4,Pennsylvania,state,NaN,98186.49315737419,98396.50663041449,98593.86288165864,98996.04493311334,99409.18660154384,...,248709.46826906633,248527.00377006107,248135.2275485898,248290.1665483627,249108.5261392972,250645.91090747225,252468.69061072348,254242.20436454364,255941.7156363109,257267.7117571273


In [2]:
median_prices_pf=pd.read_csv(
    "../Madison/Resources/Median Prices Data - Median(New) - MedianPricesofExistingDetachedHomesHistoricalData.csv.csv",
    dtype = "object",
    encoding = "utf-8"
)

median_prices_pf.head(20)

,Mon-Yr,CA,Alameda,Amador,Butte,Calaveras,Contra-Costa,Del Norte,El Dorado,Fresno,...,Sonoma,Stanislaus,Sutter,Tehama,Trinity,Tulare,Tuolumne,Ventura,Yolo,Yuba
0,Jan-90,"$194,952","$226,149",NaN,"$102,143",NaN,NaN,NaN,NaN,"$82,083",...,"$174,022",NaN,NaN,NaN,NaN,"$86,999",NaN,"$250,410",NaN,NaN
1,Feb-90,"$196,273","$219,306",NaN,"$83,333",NaN,NaN,NaN,NaN,"$87,187",...,"$180,294",NaN,NaN,NaN,NaN,"$84,375",NaN,"$235,760",NaN,NaN
2,Mar-90,"$194,856","$225,162",NaN,"$100,000",NaN,NaN,NaN,NaN,"$83,889",...,"$178,823",NaN,NaN,NaN,NaN,"$84,210",NaN,"$240,320",NaN,NaN
3,Apr-90,"$196,111","$229,333",NaN,"$108,000",NaN,NaN,NaN,NaN,"$85,428",...,"$184,117",NaN,NaN,NaN,NaN,"$88,928",NaN,"$230,620",NaN,NaN
4,May-90,"$195,281","$232,291",NaN,"$100,000",NaN,NaN,NaN,NaN,"$88,749",...,"$185,844",NaN,NaN,NaN,NaN,"$85,999",NaN,"$235,320",NaN,NaN
5,Jun-90,"$194,410","$231,250",NaN,"$123,043",NaN,NaN,NaN,NaN,"$87,999",...,"$185,333",NaN,NaN,NaN,NaN,"$88,333",NaN,"$242,310",NaN,NaN
6,Jul-90,"$193,088","$232,916",NaN,"$113,571",NaN,NaN,NaN,NaN,"$91,346",...,"$185,333",NaN,NaN,NaN,NaN,"$94,411",NaN,"$236,920",NaN,NaN
7,Aug-90,"$192,180","$228,467",NaN,"$113,333",NaN,NaN,NaN,NaN,"$95,178",...,"$180,270",NaN,NaN,NaN,NaN,"$92,083",NaN,"$231,200",NaN,NaN
8,Sep-90,"$189,979","$225,000",NaN,"$117,499",NaN,NaN,NaN,NaN,"$92,500",...,"$182,308",NaN,NaN,NaN,NaN,"$83,750",NaN,"$239,930",NaN,NaN
9,Oct-90,"$187,630","$221,093",NaN,"$127,143",NaN,NaN,NaN,NaN,"$96,891",...,"$183,414",NaN,NaN,NaN,NaN,"$90,000",NaN,"$238,220",NaN,NaN


In [3]:
#Cleaning the data 
#delete unnecessary columns 
# median_prices_pf.drop(['Amador','Calaveras','Contra-Costa','Del Norte','El Dorado','Yuba','Unnamed: 54','Central Coast',
# 'Central Valley','Far North','Inland Empire','SoCal'],axis=1)

#Only keep data from 2018-2023
median_prices_pf["Mon-Yr"] = pd.to_datetime(median_prices_pf["Mon-Yr"],format='%b-%y')
median_prices_pf=median_prices_pf.drop('CA',axis=1)
median_prices_data= median_prices_pf[median_prices_pf["Mon-Yr"] >= "2018-01-01"]
median_prices_data

,Mon-Yr,Alameda,Amador,Butte,Calaveras,Contra-Costa,Del Norte,El Dorado,Fresno,Glenn,...,Sonoma,Stanislaus,Sutter,Tehama,Trinity,Tulare,Tuolumne,Ventura,Yolo,Yuba
336,2018-01-01,"$840,000","$330,000","$314,750","$306,500","$555,000","$198,500","$480,000","$245,450","$228,500",...,"$670,000","$295,000","$284,200","$227,500",NaN,"$215,730","$265,000","$660,720","$383,000","$265,360"
337,2018-02-01,"$832,000","$318,500","$297,000","$311,500","$610,000","$216,500","$468,000","$266,440","$219,600",...,"$689,000","$300,000","$283,500","$189,000",NaN,"$225,000","$274,000","$609,000","$397,500","$269,000"
338,2018-03-01,"$955,000","$340,000","$315,000","$325,000","$655,000","$200,000","$510,000","$264,950","$244,500",...,"$685,000","$308,000","$281,000","$222,500",NaN,"$232,500","$292,500","$638,500","$422,500","$277,900"
339,2018-04-01,"$969,300","$335,000","$316,000","$340,000","$689,660","$239,000","$499,999","$260,000","$215,000",...,"$685,000","$306,000","$280,000","$210,000",NaN,"$229,250","$305,000","$665,000","$495,000","$285,000"
340,2018-05-01,"$1,025,000","$331,500","$324,100","$329,000","$687,500","$214,000","$552,000","$279,980","$230,000",...,"$698,500","$325,000","$281,000","$197,500",NaN,"$234,950","$300,000","$670,000","$476,500","$297,500"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,2023-05-01,"$1,260,000","$449,000","$436,950","$550,000","$888,000","$315,000","$660,000","$420,000","$345,000",...,"$860,000","$463,500","$439,500","$348,000","$263,250","$375,060","$424,500","$925,500","$646,730","$440,990"
401,2023-06-01,"$1,305,000","$470,000","$413,500","$459,000","$930,000","$410,000","$700,000","$429,390","$360,000",...,"$850,000","$460,000","$437,000","$362,500","$305,000","$375,000","$413,000","$927,500","$617,000","$445,000"
402,2023-07-01,"$1,260,000","$442,000","$429,000","$476,500","$900,000","$349,000","$650,000","$417,500","$372,500",...,"$850,960","$465,000","$402,000","$320,000","$320,000","$370,520","$463,500","$920,000","$625,000","$415,000"
403,2023-08-01,"$1,250,000","$490,000","$436,250","$475,000","$844,440","$382,500","$657,000","$425,000","$321,000",...,"$850,000","$465,000","$425,000","$312,050","$205,000","$372,000","$389,500","$915,000","$622,500","$449,500"


In [4]:
median_prices_data.set_index('Mon-Yr')

,Alameda,Amador,Butte,Calaveras,Contra-Costa,Del Norte,El Dorado,Fresno,Glenn,Humboldt,...,Sonoma,Stanislaus,Sutter,Tehama,Trinity,Tulare,Tuolumne,Ventura,Yolo,Yuba
Mon-Yr,,,,,,,,,,,,,,,,,,,,,
2018-01-01,"$840,000","$330,000","$314,750","$306,500","$555,000","$198,500","$480,000","$245,450","$228,500","$300,000",...,"$670,000","$295,000","$284,200","$227,500",NaN,"$215,730","$265,000","$660,720","$383,000","$265,360"
2018-02-01,"$832,000","$318,500","$297,000","$311,500","$610,000","$216,500","$468,000","$266,440","$219,600","$291,700",...,"$689,000","$300,000","$283,500","$189,000",NaN,"$225,000","$274,000","$609,000","$397,500","$269,000"
2018-03-01,"$955,000","$340,000","$315,000","$325,000","$655,000","$200,000","$510,000","$264,950","$244,500","$313,500",...,"$685,000","$308,000","$281,000","$222,500",NaN,"$232,500","$292,500","$638,500","$422,500","$277,900"
2018-04-01,"$969,300","$335,000","$316,000","$340,000","$689,660","$239,000","$499,999","$260,000","$215,000","$309,000",...,"$685,000","$306,000","$280,000","$210,000",NaN,"$229,250","$305,000","$665,000","$495,000","$285,000"
2018-05-01,"$1,025,000","$331,500","$324,100","$329,000","$687,500","$214,000","$552,000","$279,980","$230,000","$314,900",...,"$698,500","$325,000","$281,000","$197,500",NaN,"$234,950","$300,000","$670,000","$476,500","$297,500"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-01,"$1,260,000","$449,000","$436,950","$550,000","$888,000","$315,000","$660,000","$420,000","$345,000","$435,000",...,"$860,000","$463,500","$439,500","$348,000","$263,250","$375,060","$424,500","$925,500","$646,730","$440,990"
2023-06-01,"$1,305,000","$470,000","$413,500","$459,000","$930,000","$410,000","$700,000","$429,390","$360,000","$440,000",...,"$850,000","$460,000","$437,000","$362,500","$305,000","$375,000","$413,000","$927,500","$617,000","$445,000"
2023-07-01,"$1,260,000","$442,000","$429,000","$476,500","$900,000","$349,000","$650,000","$417,500","$372,500","$465,000",...,"$850,960","$465,000","$402,000","$320,000","$320,000","$370,520","$463,500","$920,000","$625,000","$415,000"


In [5]:
#Data Cleaning
#remove "$" and "," in list 






NameError: name 'median_price_2018' is not defined

In [6]:
#Store counties as a list for x-axis

median_price_counties=['Alameda', 'Amador', 'Butte', 'Calaveras',
       'Contra-Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt',
       'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles', 'Madera', 'Marin',
       'Mariposa', 'Mendocino', 'Merced', 'Mono', 'Monterey', 'Napa', 'Nevada',
       'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus',
       'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo',
       'Yuba']

#Store housing prices for 2018, 2019, 2020, 2021, 2022, and 2023

#2018 
median_price_2018 = median_prices_data.loc[[336], :].values.flatten().tolist()
del median_price_2018[0]

#2019
median_price_2019 = median_prices_data.loc[[348], :].values.flatten().tolist()
del median_price_2019[0]

#2020
median_price_2020 = median_prices_data.loc[[360], :].values.flatten().tolist()
del median_price_2020[0]

#2021
median_price_2021 = median_prices_data.loc[[372], :].values.flatten().tolist()
del median_price_2021[0]

#2022
median_price_2022 = median_prices_data.loc[[384], :].values.flatten().tolist()
del median_price_2022[0]

#2023
median_price_2023 = median_prices_data.loc[[396], :].values.flatten().tolist()
del median_price_2023[0]

In [12]:
#QUESTION 
#Data Cleaning
#how to remove "$" and "," in list ?

median_price_2018 = median_price_2018.replace('$','')

AttributeError: 'list' object has no attribute 'replace'

In [13]:
#Bar Graph - median single family home price per California county (2018)

plt.bar(median_price_counties, median_price_2018, color='r', alpha=0.5, align="edge")

plt.xlabel("CA Counties")
plt.ylabel("SFH Prices")
plt.title("Median SFH Prices per CA County (2018)")
plt.show()

In [ ]:
#Bar Graph - median price per California county (2019)


In [ ]:
#Bar Graph - median price per California county (2020)


In [ ]:
#Bar Graph - median price per California county (2021)


In [ ]:
#Bar Graph - median price per California county (2022)


In [ ]:
#Bar Graph - median price per California county (2023)


In [ ]:
percent_change_month=pd.read_csv(
    "../Madison/Resources/Sales MTM Percent Change.csv",
    dtype = "object",
    encoding = "utf-8"
)

percent_change_month.head(50)